# Machine Learning - Onderwijsdagen

Tijdens het praktische gedeelte van deze sessie gaan we aan de slag met Jupyter notebooks. Een notebook is een gebruiksvriendelijke manier om code te schrijven in de programmeertaal Python. De indeling van notebooks werkt als volgt:

- Elke rechthoekige '__cel__' in de notebook bevat een toelichting ('__Markdown__', net zoals deze cel) of Python-commando's ('__Code__'). Selecteer een cel door er 1 keer op te klikken.


- Door op de knop '__Run__' te klikken, wordt de huidige cel uitgevoerd en de volgende cel automatisch geselecteerd. Wanneer een cel commando's bevat, worden de resultaten hiervan na uitvoer onder de cel weergegeven.


Probeer op deze wijze door het notebook heen te werken. Laat je niet afschrikken door de code, in principe is alles al ingevuld.
###### Let op: wanneer er iets fout gaat en je de notebook wilt herstarten, dien je weer bovenaan opnieuw te beginnen!

Doel van deze notebook is het trainen van 10 __perceptrons__. Kort gezegd is een perceptron een simpel neuraal netwerk, bestaande uit een $X$ aantal __inputs__ - de invoer. Alle inputs zijn verbonden met een __output__ - de uitvoer. De verbindingen tussen $X$ en deze outputs, die de __gewichten__ heten, worden getraind om te zorgen dat de juiste output wordt geactiveerd wanneer de input bepaalde patronen bevat.

Tijdens dit experiment trainen we 10 perceptrons tegelijkertijd op dezelfde invoer, met als doel __automatisch handgeschreven nummers te herkennen__ ('0' tot '9'). Elke perceptron krijgt als taak om een van de getallen te herkennen; afhankelijk van hoe 'sterk' het netwerk een getal denkt te herkennen wordt de perceptron van het respectievelijke getal geactiveerd. Elke output genereert op deze manier een 'kans'-waarde. Hoe hoger deze waarde, hoe zekerder het netwerk is van zijn classificatie. Bij een slordig geschreven 8 kan het zijn dat het netwerk zowel een 3 en een 8 als output herkent. De output met de hoogste waarde is de uiteindelijke classificatie van de input.

<img src="images/Perceptron2.png" alt="Perceptron visualization" style="width: 400px;"/>

>_Illustratie van de 10 perceptrons._

Laten we starten met het importeren van de vereiste plugins. Voer de cel uit en controleer of er een boodschap onder de cel verschijnt.

In [ ]:
import os, cv2, keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers
from keras.utils import to_categorical
keras.initializers.RandomNormal(seed=42)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)
print("Klaar!")

We gaan de perceptrons trainen op de breed toegepaste __MNIST-dataset__, die tienduizenden afbeeldingen van handgeschreven nummers bevat. Elk plaatje bevat een zwart-wit nummer met een resolutie van 28 x 28 pixels. Ieder plaatje is van een label voorzien, en op die manier toegewezen aan een van de 10 categorieen ('0', '1', .... '9').

Om plaatjes te kunnen verwerken in het neurale netwerk, worden ze '__platgeslagen__'. De afbeelding van 28 x 28 pixels wordt omgezet naar een rij van 28*28=784 getallen. De $X$-waarde van ons neurale netwerk is dus 784 inputs, gelijk aan de hoeveelheid pixels die het plaatje bevat.

<img src="images/Perceptron-MNIST.png" alt="Perceptron visualization 2" style="width: 500px;"/>

>_Verwerking van een plaatje in het neurale netwerk (vereenvoudigde weergave)._

De MNIST-dataset bestaat uit een __training- en test-dataset__. Een vuistregel voor experimenten met neurale netwerken is om een deel van de beschikbare data apart te zetten in een test-dataset, geschikt voor validatiedoeleinden. Op deze manier kunnen we de nauwkeurigheid van ons neurale netwerk bij gebruik van nieuwe data valideren. Uiteraard zijn alletwee de datasets ook van een lijst met corresponderende __labels__ voorzien.

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print('MNIST data is geimporteerd.')

De perceptrons worden straks getraind op de plaatjes uit de __training-dataset__, door ze samen met de __training-labels__ door het neurale netwerk te laten verwerken. 

Laten we bekijken hoe de training-dataset in elkaar zit. Als het goed is laat de code hieronder zien dat er __60.000 plaatjes__ in de dataset zitten, met een resolutie van __28 x 28 pixels__.

In [ ]:
train_images.shape

Klopt helemaal! Laten we eens kijken hoe de __eerste 8 plaatjes__ uit de dataset eruit zien.

In [ ]:
plt.subplot(241)
plt.imshow(train_images[10].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(242)
plt.imshow(train_images[1].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(243)
plt.imshow(train_images[2].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(244)
plt.imshow(train_images[3].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(245)
plt.imshow(train_images[4].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(246)
plt.imshow(train_images[5].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(247)
plt.imshow(train_images[6].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(248)
plt.imshow(train_images[7].reshape(28,28), cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

Dat ziet er goed uit. We hebben natuurlijk ook de __labels__ bij de plaatjes nodig. Het aantal labels zou gelijk moeten zijn aan het aantal plaatjes (__60.000__).

In [ ]:
len(train_labels)

Hoeveel __unieke labels__ zijn er? Dit zouden de cijfers '0' tot en met '9' moeten zijn.

In [ ]:
print(np.unique(train_labels))

Zijn de labels (ongeveer) __gebalanceerd__? Klopt het dat er ongeveer evenveel plaatjes per categorie in de training-dataset zitten?

In [ ]:
plt.hist(train_labels)

plt.title("Train Label Histogram")
plt.xlabel("Label")
plt.ylabel("Aantal")
plt.xticks(np.arange(10))
fig = plt.gcf()

We kunnen hetzelfde doen voor de __test-dataset__. Deze zou uit 10.000 plaatjes moeten bestaan.

In [ ]:
test_images.shape

Met 10.000 __labels__...

In [ ]:
len(test_labels)

En dezelfde __unieke labels__ (cijfers '0' tot en met '9')...

In [ ]:
print(np.unique(test_labels))

Met een (min of meer) __gelijke verdeling__.

In [ ]:
plt.hist(test_labels)

plt.title("Test Label Histogram")
plt.xlabel("Label")
plt.ylabel("Aantal")
plt.xticks(np.arange(10))
fig = plt.gcf()

-----
### <b><center>Pauzeer hier</center></b>

-----

Ieder neuraal netwerk is opgebouwd uit __lagen__. Iedere laag in een neuraal netwerk zou je kunnen zien als een filter waardoor de data wordt verwerkt. In ons geval worden er plaatjes van handgeschreven nummers ingevoerd, deze worden door het netwerk verwerkt, en vervolgens uitgevoerd in een voor het netwerk bruikbare vorm.

De lagen filteren __representaties__ uit uit de ingevoerde data -- hopenlijk representaties die kunnen helpen bij het oplossen van een probleem (in dit geval het correct herkennen van onze handgeschreven nummers).

Het grootste deel van deep learning bestaat uit het bouwen van neurale netwerken, die bestaan uit vele varianten van deze lagen. De netwerken voeren een soort '__data-destillatie__' uit, door keer op keer representaties uit de data te filteren.

Laten we beginnen met het opstellen van ons eigen neurale netwerk! Het netwerk bestaat uit:

- Een __invoerlaag__ met 28*28=784 inputs (elke pixel uit een plaatje is een input).


- Een __uitvoerlaag__ met 10 outputs, voor elk getal een enkele output ('0', '1', ...., '9'). De output met de hoogste waarde is de classificatie die het netwerk aan het plaatje toekent.

In [ ]:
network = models.Sequential()
network.add(layers.Dense(10, use_bias='true', activation='softmax', input_shape=(28 * 28,)))

print('Lagen toegevoed.')

De lagen zijn nu toegevoegd, maar we zijn er nog niet. Om het neurale netwerk klaar te maken voor gebruik moeten er nog 3 zaken geregeld worden, daarna kan het netwerk worden __samengesteld__.

* Een __loss-functie__: deze zorgt ervoor dat het netwerk een idee heeft van hoe 'goed' of 'slecht' het scoort op de training-dataset. Op deze manier kan het netwerk zichzelf bijsturen.


* De __optimizer__: dit mechanisme zorgt ervoor dat het netwerk zichzelf in stappen kan bijwerken op basis van de ingevoerde data en de loss-functie.


* De waarden die bijgehouden gaan worden tijdens het trainen en testen. In dit geval houden we alleen rekening met de __nauwkeurigheid (accuracy)__ van het netwerk, bestaande uit het aantal plaatjes dat correct is geclassificeerd.

In [ ]:
keras.optimizers.SGD(lr=0.01)

network.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

networksave = network.get_weights()

print('Netwerk samengesteld.')

Voordat het daadwerkelijke trainen van het netwerk kan beginnen, moet de data __omgevormd__ worden naar een formaat dat het neurale netwerk verwacht. Alle plaatjes worden __'platgeslagen'__ naar een lijst van 784 cijfers.

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print('Data-verwerking afgerond.')

Het trainen van het netwerk gaat als volgt in zijn werk:

- Eerst voeren we de __training dataset__ in binnen het neurale netwerk (`train_images` en de bijbehorende `train_labels`). De perceptrons in het neurale netwerk leren op die manier via de __gewichten__ een associatie aan tussen de plaatjes en de labels.


- Daarna passen we de __test-dataset__ op het netwerk toe, door de `test_images` in het netwerk te laten verwerken. Dan __valideren__ we of de voorspellingen uit het netwerk overeenkomen met de labels uit `test_labels`.

Het __trainen__ wordt gestart door de `fit`-functie van het netwerk aan te roepen. Het aantal `epochs` staat voor het aantal keer dat het netwerk door de training-dataset itereert. De `batch_size` geeft aan hoeveel groepen van plaatjes uit de dataset door het netwerk worden gebruikt om zichzelf bij te sturen. Hoe hoger de batch size, hoe sneller het netwerk is met trainen, maar daardoor gaat de nauwkeurigheid ook omlaag.

>_Extra: Experimenteer met verschillende waarden voor de batch size en het aantal epochs, om te zien op welke manier dit invloed heeft op de nauwkeurigheid van het netwerk. Vervang de cijfers achter_ `epochs=` _en_ `batch_size=` _in de onderstaande cel om deze aan te passen._

In [ ]:
network.set_weights(networksave)
network.fit(train_images, train_labels, epochs=20, batch_size=20)

Tijdens het trainen worden er twee getallen weergegeven: de `loss` van het netwerk over de training-dataset, en `acc`, de nauwkeurigheid die het neurale netwerk behaalt op de training data:

- De __loss__ zou zo laag mogelijk moeten zijn, deze waarde representeert de fouten die het netwerk maakt tijdens classificatie.


- De __nauwkeurigheidswaarde__ laat zien welk percentage van de training-dataset correct geclassificeerd is. Let wel op dat het daarbij alleen nog maar om de __training-dataset__ gaat!

Om erachter te komen hoe goed het netwerk functioneert bij verwerking van nieuwe data, evalueren we de naukeurigheid op de __test-dataset__.

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('Nauwkeurigheid netwerk op de test-dataset:', test_acc)

We zien dat de nauwkeurigheidsscore voor de test-dataset ongeveer gelijk ligt met de score voor de training-dataset. Dit betekent dat het neurale netwerk goed __generaliseerbaar__ is. De praktijk laat dit ook zien; in de jaren 90 werd een soortgelijk neuraal netwerk gebruikt om postcodes op brieven automatisch te herkennen.

------
### <b><center>Pauzeer hier</center></b>

------

Een interessante eigenschap van perceptrons (en vele andere typen neurale netwerken), is dat we de __interne staat__ van een perceptron na training kunnen __visualiseren__.

Hieronder bouwen we de percpetrons opnieuw op in een matrix van 28 x 28 pixels, zodat we deze visueel als een afbeelding kunnen interpreteren.

In [ ]:
W1 = network.layers[0].get_weights()[0]
WW = W1.reshape(28,28,10)

plt.subplot(2,5,1)
plt.imshow(WW[:,:,0], cmap=plt.get_cmap('gray'))
plt.title('output 0')
plt.subplot(2,5,2)
plt.imshow(WW[:,:,1], cmap=plt.get_cmap('gray'))
plt.title('output 1')
plt.subplot(2,5,3)
plt.imshow(WW[:,:,2], cmap=plt.get_cmap('gray'))
plt.title('output 2')
plt.subplot(2,5,4)
plt.imshow(WW[:,:,3], cmap=plt.get_cmap('gray'))
plt.title('output 3')
plt.subplot(2,5,5)
plt.imshow(WW[:,:,4], cmap=plt.get_cmap('gray'))
plt.title('output 4')
plt.subplot(2,5,6)
plt.imshow(WW[:,:,5], cmap=plt.get_cmap('gray'))
plt.title('output 5')
plt.subplot(2,5,7)
plt.imshow(WW[:,:,6], cmap=plt.get_cmap('gray'))
plt.title('output 6')
plt.subplot(2,5,8)
plt.imshow(WW[:,:,7], cmap=plt.get_cmap('gray'))
plt.title('output 7')
plt.subplot(2,5,9)
plt.imshow(WW[:,:,8], cmap=plt.get_cmap('gray'))
plt.title('output 8')
plt.subplot(2,5,10)
plt.imshow(WW[:,:,9], cmap=plt.get_cmap('gray'))
plt.title('output 9')
# show the plot
plt.show()

Wat valt je op wanneer je de 10 visualisaties bekijkt? Waarom laat iedere perceptron dit specifieke patroon zien?

-------
### <b><center>Pauzeer hier</center></b>

-------

>_Extra: Door de onderstaande cel uit te voeren, wordt een iets geavanceerdere versie van het netwerk dat we net hebben gebouwd weergegeven. Experimenteer met het tekenen van cijfers, om te zien of de vorm of positie van invloed zijn op de voorspellingen van het netwerk._

In [ ]:
%%html
<iframe src="http://scs.ryerson.ca/~aharley/vis/fc/" width="990" height="550"></iframe>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Deze Jupyter notebook is gebaseerd op https://github.com/fchollet/deep-learning-with-python-notebooks.